###  Using API to run the simulation

We use a strongly defined HTTP API protocol to communicate with a C12 backend (get and receive the data). This tutorial will briefly present basic API calls that can be used for communication with the C12 backend. Our APIs are the low-level access to our simulator, and could be made compatible with other quantum computer frameworks and interfaces.

To run each API, we need to pass the unique bearer token that will be used to verify the user's right to access C12's system.

For communication with C12's system, it is possible to use the API library that encapsulates the HTTP communication with the C12 system using the python request module.

To start using it, we need to create an instance of the Request class, as shown in the following code snippet.

In [8]:
# Installing the c12simulator-clients package (for more details see jupyter 1)
import os
from c12_callisto_clients.api.client import Request

TOKEN = os.getenv("C12_TOKEN")


# Create the request instance
# Constructor of the Request class also accepts the verbose parameter, which can be use for more detailed output of the methods.

request = Request(auth_token=TOKEN, verbose=False)

#### Run the simulation

The method `start_job()` is used to start the simulation. It has the following arguments:
- `qasm_str`: QASM string with quantum circuit
- `shots`:  Number of trials for the simulation
- `result`: what is desired output (statevector, counts, density_matrix)
- `backend_name`: the name of the backend to run on

In [9]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)


qasm_str = circuit.qasm()
res = request.start_job(
    qasm_str=qasm_str, shots=100, result="density_matrix", backend_name="c12sim-iswap"
)

# The function returns the UUID and QasmStr of started job or HTTP error in case of problems
print(res)
job_uuid = res[0]


('176f1988-e3a8-4b8b-8323-a4684d62943a', 'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }\nqreg q[2];\nry(-pi/2) q[1];\niswap q[0],q[1];\nrx(pi) q[0];\nry(-pi/2) q[1];\niswap q[0],q[1];\nry(-pi/2) q[0];\nrz(-pi/2) q[0];\nrz(pi/2) q[1];\nry(-pi/2) q[1];\n')


#### Get the results of a simulation

To get the simulation results, we can use the `get_job_result()` method that accepts one mandatory argument (job UUID string) and two optional ones.
- `timeout`, which represents the number of seconds that the function will wait for job completion
- `wait`, number of seconds between each check if the job is finished

If we do not specify a timeout argument the function will run until the result is obtained

In [10]:
job_result = request.get_job_result(job_uuid, timeout=120, wait=5)
print(job_result)

{'status': 'FINISHED', 'results': {'density_matrix': [['(0.4990637768053288+1.3551869826774539e-21j)', '(0.00010967621893653856+0.0001964744471174757j)', '(-1.2293983629452221e-05+0.0002125517091276997j)', '(0.4983895890488784-0.00020246473017107892j)'], ['(0.00010967621893652468-0.00019647444711747572j)', '(0.0006796467390991628-2.032942735425706e-21j)', '(5.139379161905877e-05+4.017392210456757e-05j)', '(1.3313268045392346e-05-3.129665779283896e-05j)'], ['(-1.2293983629424465e-05-0.0002125517091276997j)', '(5.139379161905877e-05-4.017392210456757e-05j)', '(0.0010610935097781216+1.3551890535914957e-21j)', '(7.144323329608127e-05+0.00012217004103256572j)'], ['(0.4983895890488784+0.00020246473017107892j)', '(1.3313268045378468e-05+3.129665779283896e-05j)', '(7.144323329606739e-05-0.00012217004103256575j)', '(0.4991954829457991-2.0329406645084996e-21j)']]}, 'errors': ''}


#### Get the status of the job

In [11]:
status = request.get_job_status(job_uuid=job_uuid)
print(status)

finished


#### Get all jobs

Method `get_user_jobs()` can be used for obtaining information about all the jobs that have been run on the system for the user whose token is passed to the Request instance.

The method has two arguments:
- `limit`: an integer that represents the number of results to obtain
- `offset`: an integer that represents the offset from the first result

Using these two values, one can obtain the pagination, where the offset represents a page and limits the number of results per page. Method returns an array with results.


In [12]:
number_of_results = 5  # Limit parameter, number of records to retrieve
offset = 0
count = 1
while True:
    jobs = request.get_user_jobs(limit=number_of_results, offset=offset)

    for job in jobs:
        print(f"{count:}")
        print(job)
        count += 1

    if len(jobs) < number_of_results:
        break

    offset += number_of_results

1
{'uuid': 'a7fadc44-22f7-42a1-9056-cd05f510a279', 'time': '2023-09-19T15:37:51.654241', 'status': 'finished', 'options': {'shots': 10000, 'result': 'counts,statevector', 'basis': 'iswap'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }\nqreg q[2];\nry(-pi/2) q[1];\niswap q[0],q[1];\nrx(pi) q[0];\nry(-pi/2) q[1];\niswap q[0],q[1];\nry(-pi/2) q[0];\nrz(-pi/2) q[0];\nrz(pi/2) q[1];\nry(-pi/2) q[1];\n', 'task_orig': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nh q[0];\ncx q[0],q[1];\n', 'errors': '4', 'result': '4'}
2
{'uuid': '7b646f18-198d-40d7-a3f1-cb87cb6cc7a7', 'time': '2023-09-19T15:38:08.722888', 'status': 'finished', 'options': {'shots': 10000, 'result': 'counts,statevector', 'basis': 'iswap'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }\nqreg q[2];\nry(-pi/2) q[1];\niswap q[0],q[1];\nrx(pi) q[0];\nry(-pi/2) q[1];\niswap q[0],q[1];\nry(-pi/2) q

#### Get information about a specific job

Using the method `get_job()` it is possible to retrieve the information about the specific job with a given UUID.

In [13]:
job_data = request.get_job(job_uuid)
print(job_data)

{'uuid': '176f1988-e3a8-4b8b-8323-a4684d62943a', 'time': '2023-09-20T08:18:20.655037', 'status': 'finished', 'options': {'shots': 100, 'result': 'density_matrix', 'basis': 'iswap'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }\nqreg q[2];\nry(-pi/2) q[1];\niswap q[0],q[1];\nrx(pi) q[0];\nry(-pi/2) q[1];\niswap q[0],q[1];\nry(-pi/2) q[0];\nrz(-pi/2) q[0];\nrz(pi/2) q[1];\nry(-pi/2) q[1];\n', 'task_orig': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nh q[0];\ncx q[0],q[1];\n', 'errors': '', 'result': {'statevector': ['(0.4990637768053288+1.3551869826774539e-21j)', '(0.0006796467390991628-2.032942735425706e-21j)', '(0.0010610935097781216+1.3551890535914957e-21j)', '(0.4991954829457991-2.0329406645084996e-21j)'], 'density_matrix': [['(0.4990637768053288+1.3551869826774539e-21j)', '(0.00010967621893653856+0.0001964744471174757j)', '(-1.2293983629452221e-05+0.0002125517091276997j)', '(0.4983895890488784-0.000202464730171